# Homework for week 2

## Question 1: [IPO] Withdrawn IPOs by Company Type

In [66]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from io import StringIO

import string

In [67]:
url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url)
#response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
html_io = StringIO(response.text)
df_ipo = pd.read_html(html_io)

In [68]:
df_ipo = df_ipo[0]

In [69]:
df_ipo.shape

(100, 4)

In [70]:
df_ipo.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [71]:
def company_class(name):

     # Convert to lowercase and remove punctuation
    name = name.lower()
    name = name.translate(str.maketrans('', '', string.punctuation))
    words = name.split()

    if 'acquisition' in words and ('corp' in words or 'corporation' in words):
        return 'Acq.Corp'
    elif 'inc' in words or 'incorporated' in words :
        return 'Inc'
    elif 'group' in words:
        return 'Group'
    elif 'holdings' in words:
        return 'Holdings'
    elif 'ltd' in words or 'limited' in words:
        return 'Ltd'
    else:
        return 'Other'

In [72]:
df_ipo["Company Class"] = df_ipo["Company Name"].apply(company_class)


In [73]:
df_ipo['Company Class'].value_counts()

Company Class
Inc         51
Acq.Corp    21
Ltd         12
Other        6
Holdings     6
Group        4
Name: count, dtype: int64

In [ ]:
#average price function
def average_price(price_str):
    if pd.isna(price_str) or price_str == '-':
        return None
    # Remove dollar sign and split by hyphen if present
    price_str = price_str.replace('$', '').strip()
    if '-' in price_str:
        prices = price_str.split('-')
        return np.mean([float(p.strip()) for p in prices])
    else:
        return float(price_str)

In [75]:
df_ipo["Avg. price"] = df_ipo['Price Range'].apply(average_price)
df_ipo

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [ ]:
#Convert Shares Offered to numeric, clean missing or invalid values
df_ipo['Shares Offered'] = pd.to_numeric(df_ipo['Shares Offered'], errors='coerce')


In [77]:
#Withdrawn Value = Shares Offered * Avg Price 
df_ipo['Withdrawn Value'] = df_ipo['Shares Offered'] * df_ipo['Avg. price']

In [79]:
df_ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Symbol           100 non-null    object 
 1   Company Name     100 non-null    object 
 2   Price Range      100 non-null    object 
 3   Shares Offered   72 non-null     float64
 4   Company Class    100 non-null    object 
 5   Avg. price       73 non-null     float64
 6   Withdrawn Value  71 non-null     float64
dtypes: float64(3), object(4)
memory usage: 5.6+ KB


In [80]:
#Group by Company Class and calculate total withdrawn value
df_ipo.groupby('Company Class')['Withdrawn Value'].sum().sort_values(ascending=False).reset_index()


,Company Class,Withdrawn Value
0,Acq.Corp,4.021000e+09
1,Inc,2.257164e+09
2,Other,7.679200e+08
3,Ltd,3.217346e+08
4,Holdings,3.030000e+08
5,Group,3.378750e+07


## Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

In [161]:
# IPO data for 2024
url_2024 = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url_2024)  
html_io = StringIO(response.text)
df_ipo_2024 = pd.read_html(html_io)

In [162]:
df_ipo_2024 = df_ipo_2024[0]
df_ipo_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$3.40,-13.75%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$20.29,423.75%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.56,-85.05%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.26,2.48%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.45,4.50%


In [163]:
#Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024)
df_ipo_2024['IPO Date'] = pd.to_datetime(df_ipo_2024['IPO Date'])
df_ipo_2024 = df_ipo_2024[df_ipo_2024['IPO Date'] < '2024-06-01'].reset_index(drop=True)

In [164]:
df_ipo_2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-05-31,NAKA,"Kindly MD, Inc.",-,$18.29,-
1,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$35.01,105.59%
2,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$21.00,7.36%
3,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.52,5.20%
4,2024-05-15,JDZG,JIADE Limited,$4.00,$0.30,-92.75%
...,...,...,...,...,...,...
72,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.08,-81.67%
73,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$13.85,-15.81%
74,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$1.80,-56.00%
75,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$19.62,-5.48%


In [165]:
ALL_TICKERS = df_ipo_2024['Symbol'].unique()
len(ALL_TICKERS)

77

In [166]:
import yfinance as yf
import time

In [167]:
# download daily stock data
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [168]:
stocks_df['Ticker'].nunique()

77

In [169]:
stocks_df.head().transpose()

,0,1,2,3,4
Open,4.0,2.99,2.53,2.91,2.94
High,4.2,3.11,3.11,3.09,2.94
Low,2.8,2.35,2.41,2.6,2.41
Close,3.02,2.66,2.92,2.73,2.69
Volume,440600,147300,73800,51100,56500
Dividends,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,NAKA,NAKA,NAKA,NAKA
Year,2024,2024,2024,2024,2024
Month,5,6,6,6,6


In [170]:
#Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:

stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [171]:
#Filter the DataFrame to keep data only for the trading day '2025-06-06'
stocks_df_filtered = stocks_df[stocks_df['Date'].astype(str) == '2025-06-06'].reset_index(drop=True)

In [172]:
stocks_df_filtered[['growth_252d', 'Sharpe']].describe()

,growth_252d,Sharpe
count,73.000000,73.000000
mean,1.227946,0.297523
std,1.480238,0.523190
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.083768
75%,1.446667,0.331967
max,8.097413,2.835668


In [173]:
#top 10 companies when sorting by growth_252d versus sorting by Sharpe
top_growth = stocks_df_filtered[['Ticker', 'growth_252d', 'Sharpe']].sort_values(by='growth_252d', ascending=False).head(10)
top_sharpe = stocks_df_filtered[['Ticker', 'Sharpe', 'growth_252d']].sort_values(by='Sharpe', ascending=False).head(10) 


In [174]:
top_growth

,Ticker,growth_252d,Sharpe
68,JL,8.097413,0.566222
76,ROMA,6.156406,0.486840
0,NAKA,5.438356,0.039408
50,UMAC,4.966533,0.421306
11,NNE,4.655224,0.080707
17,RBRK,3.184065,0.019051
57,AHR,2.483097,0.112576
61,AS,2.478203,0.028026
26,SUPX,2.346065,0.266389
16,MRX,2.300384,0.076540


In [175]:
top_sharpe

,Ticker,Sharpe,growth_252d
39,BKHA,2.835668,1.045881
70,JVSA,2.041531,1.071076
58,LEGT,1.940267,1.049407
37,IBAC,1.637119,1.044611
18,NCI,1.181375,0.572687
51,HLXB,1.123493,1.061404
30,MNDR,0.974234,1.030769
48,DYCQ,0.969321,1.059863
43,INTJ,0.744512,0.661386
68,JL,0.566222,8.097413


# Question 3: [IPO] ‘Fixed Months Holding Strategy’

In [176]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
0,4.00,4.2000,2.80,3.02,440600,0.0,0.0,NAKA,2024,5,...,NaN,NaN,0.784768,NaN,NaN,0,0.463576,NaN,0,NaN
1,2.99,3.1100,2.35,2.66,147300,0.0,0.0,NAKA,2024,6,...,NaN,NaN,0.883459,NaN,NaN,0,0.285714,NaN,0,NaN
2,2.53,3.1100,2.41,2.92,73800,0.0,0.0,NAKA,2024,6,...,NaN,NaN,0.955479,NaN,NaN,0,0.239726,NaN,0,NaN
3,2.91,3.0900,2.60,2.73,51100,0.0,0.0,NAKA,2024,6,...,NaN,NaN,1.051282,NaN,NaN,0,0.179487,NaN,1,NaN
4,2.94,2.9400,2.41,2.69,56500,0.0,0.0,NAKA,2024,6,...,NaN,NaN,1.111524,NaN,NaN,0,0.197026,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23408,3.74,4.1350,3.36,3.57,264600,0.0,0.0,ROMA,2025,6,...,6.144579,NaN,NaN,3.0361,2.54155,1,0.217087,12.173744,0,0.501044
23409,3.65,3.9500,3.63,3.70,84400,0.0,0.0,ROMA,2025,6,...,6.156406,NaN,NaN,3.2071,2.62555,1,0.086486,12.553202,0,0.486840
23410,3.75,3.9550,2.60,2.86,381700,0.0,0.0,ROMA,2025,6,...,4.627832,NaN,NaN,3.2630,2.66355,1,0.473776,12.238182,0,0.374470
23411,2.85,3.3100,2.77,2.77,52500,0.0,0.0,ROMA,2025,6,...,5.368217,NaN,NaN,3.2600,2.70055,1,0.194946,11.810360,0,0.450724


In [177]:
# Calculate future growth for each stock over the next 1 to 12 months

for ticker in stocks_df['Ticker'].unique():
    mask = stocks_df['Ticker'] == ticker
    for i in range(1, 13):  # 1 to 12 months
        col_name = f'growth_future_{i}m'
        # Calculate growth over future 21*i trading days
        stocks_df.loc[mask, col_name] = (
            stocks_df.loc[mask, 'Close'].shift(-21 * i) / stocks_df.loc[mask, 'Close']
        )

In [178]:
#Determine the first trading day (min_date) for each ticker.

min_date  = stocks_df.groupby('Ticker')['Date'].min().reset_index()
min_date 

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
72,YIBO,2024-01-25
73,YYGH,2024-04-22
74,ZBAO,2024-04-02
75,ZK,2024-05-10


In [179]:
min_date_df = pd.merge(min_date, stocks_df[['Ticker', 'Date','growth_future_1m',
       'growth_future_2m', 'growth_future_3m', 'growth_future_4m',
       'growth_future_5m', 'growth_future_6m', 'growth_future_7m',
       'growth_future_8m', 'growth_future_9m', 'growth_future_10m',
       'growth_future_11m', 'growth_future_12m']], on=['Ticker', 'Date'], how='inner')

In [184]:
min_date_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
growth_future_1m,77.0,0.926390,0.342767,0.098947,0.775236,0.977000,1.047796,2.646505
growth_future_2m,77.0,0.936866,0.568481,0.073800,0.680193,1.000000,1.139076,4.874759
growth_future_3m,77.0,0.834120,0.410362,0.060947,0.508139,0.927500,1.075676,2.040000
growth_future_4m,77.0,0.822711,0.401473,0.045368,0.515385,0.909091,1.125316,1.605000
growth_future_5m,77.0,0.802881,0.486716,0.054109,0.445378,0.821092,1.017822,3.213873
growth_future_6m,77.0,0.864347,0.649114,0.061432,0.387417,0.802239,1.103896,3.670520
growth_future_7m,77.0,0.846270,0.706456,0.048274,0.324100,0.844875,1.150909,5.122350
growth_future_8m,77.0,0.829971,0.754255,0.043103,0.225974,0.812109,1.074739,5.171484
growth_future_9m,76.0,0.889751,0.926304,0.033144,0.267208,0.833357,1.043671,6.764933
growth_future_10m,76.0,0.913269,0.900344,0.041357,0.248485,0.772592,1.194894,5.352601


## Question 4: [Strategy] Simple RSI-Based Trading Strategy